In [1]:
#imports
import numpy as np
import astropy 
from astropy.io import ascii
from astropy.table import Table, Column, join, vstack
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
#change column names in koenig selected strips, write new file with key columns
start = 20
end = 31

while start <= end:
    filename = '../../KoenigSelectedCatalogs/Orion/SelectionV2/part'+str(start)+'.txt'
    data = ascii.read(filename)
    data.rename_column('col2', 'ra')
    data.rename_column('col3', 'dec')
    data.rename_column('col290', 'h_m_2mass')
    data.rename_column('col292', 'k_m_2mass')
    data.rename_column('col288', 'j_m_2mass')
    data.rename_column('col289', 'j_m_2mass_err')
    data.rename_column('col291', 'h_m_2mass_err')
    data.rename_column('col293', 'k_m_2mass_err')
    
    data.rename_column('col17', 'w1_m')
    data.rename_column('col21', 'w2_m')
    data.rename_column('col25', 'w3_m')
    data.rename_column('col29', 'w4_m')
    
    data.rename_column('col18', 'w1_sig')
    data.rename_column('col22', 'w2_sig')
    data.rename_column('col26', 'w3_sig')
    data.rename_column('col30', 'w4_sig')
    
    
    newFilename = '../../KoenigSelectedCatalogs/Orion/SelectionV2/part'+str(start)+'_wise.txt'
    ascii.write(data, newFilename, 
                include_names = ('ra', 'dec','w1_m','w1_sig', 'w2_m','w2_sig','w3_m','w3_sig','w4_m','w4_sig','j_m_2mass','j_m_2mass_err','h_m_2mass','h_m_2mass_err','k_m_2mass', 'k_m_2mass_err', 'Class', 'GeneralSelection'))
    start = start+1

In [7]:
#*****ONLY FOR TAURUS*****
#add catalog flag to esplin catalog, write new file with key columns
data = ascii.read('../../WISEandGALEXmatchCatalogs/WISEcatalogWith2MASS.txt')
colCat = Column(name = 'Catalog', dtype = 'a5', length = len(data))
data.add_column(colCat)
for i in range(len(data)):
    data['Catalog'][i] = 'E'
data.rename_column('j_msig_2mass', 'j_m_2mass_err')
data.rename_column('h_msig_2mass', 'h_m_2mass_err')
data.rename_column('k_msig_2mass', 'k_m_2mass_err')
ascii.write(data, '../../KoenigSelectedCatalogs/Taurus/EsplinFewColumns.txt', 
            include_names = ('ra', 'dec', 'j_m_2mass','j_m_2mass_err','h_m_2mass','h_m_2mass_err','k_m_2mass', 'k_m_2mass_err', 'Catalog'))


In [3]:
#read in all necessary files (as created above) to combine
part20 = ascii.read('../../KoenigSelectedCatalogs/Orion/SelectionV2/part20_wise.txt')
part21 = ascii.read('../../KoenigSelectedCatalogs/Orion/SelectionV2/part21_wise.txt')
part22 = ascii.read('../../KoenigSelectedCatalogs/Orion/SelectionV2/part22_wise.txt')
part23 = ascii.read('../../KoenigSelectedCatalogs/Orion/SelectionV2/part23_wise.txt')
part24 = ascii.read('../../KoenigSelectedCatalogs/Orion/SelectionV2/part24_wise.txt')
part25 = ascii.read('../../KoenigSelectedCatalogs/Orion/SelectionV2/part25_wise.txt')
part26 = ascii.read('../../KoenigSelectedCatalogs/Orion/SelectionV2/part26_wise.txt')
part27 = ascii.read('../../KoenigSelectedCatalogs/Orion/SelectionV2/part27_wise.txt')
part28 = ascii.read('../../KoenigSelectedCatalogs/Orion/SelectionV2/part28_wise.txt')
part29 = ascii.read('../../KoenigSelectedCatalogs/Orion/SelectionV2/part29_wise.txt')
part30 = ascii.read('../../KoenigSelectedCatalogs/Orion/SelectionV2/part30_wise.txt')
part31 = ascii.read('../../KoenigSelectedCatalogs/Orion/SelectionV2/part31_wise.txt')



#combine all the koenig files
koenig1 = join(part20, part21, join_type='outer')
koenig2 = join(koenig1, part22, join_type='outer')
koenig3 = join(koenig2, part23, join_type='outer')
koenig4 = join(koenig3, part24, join_type='outer')
koenig5 = join(koenig4, part25, join_type='outer')
koenig6 = join(koenig5, part26, join_type='outer')
koenig7 = join(koenig6, part27, join_type='outer')
koenig8 = join(koenig7, part28, join_type='outer')
koenig9 = join(koenig8, part29, join_type='outer')
koenig10 = join(koenig9, part30, join_type='outer')

finalTable = join(koenig10, part31, join_type='outer')


In [4]:
#write the final combined table to a file -- This is your final Koenig selected catalog!
ascii.write(finalTable, 
            '../../KoenigSelectedCatalogs/Orion/SelectionV2/CottleIR_Excess_Selection_v2.txt', 
            fill_values=[(ascii.masked, 'null')]) #fill_values inserts dummy variables 



In [5]:
#write file containing only sources that meet the APOGEE limits 7 < H < 13
combinedTable = ascii.read('../../KoenigSelectedCatalogs/Orion/SelectionV2/CottleIR_Excess_Selection_v2.txt')
newTable = Table()
for name in combinedTable.colnames:
    col = Column(name = name, dtype = combinedTable[name].dtype)
    newTable.add_column(col)
    
for i in range(len(combinedTable)):
    if combinedTable['h_m_2mass'][i] > 7.0 and combinedTable['h_m_2mass'][i] < 13.0:
        newTable.add_row(combinedTable[i])

In [6]:
ascii.write(newTable, '../../KoenigSelectedCatalogs/Orion/SelectionV2/CottleIR_Excess_Selection_v2_APOGEELimits.txt')